In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import re # Để dùng biểu thức chính quy
import pickle # Để lưu Document objects

# Hàm để đọc nội dung từ file
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return text

# Đường dẫn đến file văn bản của bạn
input_file_path = 'DSTS.txt'
novel_text = read_text_file(input_file_path)

# --- Cấu hình RecursiveCharacterTextSplitter ---
# Các giá trị này có thể cần điều chỉnh sau khi xem kết quả
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Kích thước mong muốn của mỗi chunk (số ký tự)
    chunk_overlap=150,    # Số ký tự chồng lấn giữa các chunk
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""], # Ưu tiên chia theo đoạn, rồi dòng, rồi câu
    is_separator_regex=False
)

# --- Thực hiện chia chunk và tạo Document objects ---
raw_chunks = text_splitter.split_text(novel_text)
documents = []
current_chapter = None # Biến để lưu trữ chương hiện tại

print(f"Đang xử lý và tạo Document objects cho {len(raw_chunks)} chunks thô...")

for i, chunk_content in enumerate(raw_chunks):
    # Cố gắng xác định chương cho chunk này dựa vào sự xuất hiện của "Chương X"
    # Điều này giả định rằng tiêu đề chương là thứ duy nhất có dạng "Chương [số]"
    # và nó xuất hiện ở đầu một đoạn văn/chunk.
    chapter_match_title = re.match(r"Chương\s+(\d+):?\s*([^\n]+)", chunk_content) # Tìm tiêu đề đầy đủ
    chapter_match_simple = re.match(r"Chương\s+(\d+)", chunk_content) # Tìm "Chương X" đơn giản

    if chapter_match_title:
        current_chapter_number = int(chapter_match_title.group(1))
        current_chapter_title = chapter_match_title.group(2).strip()
        current_chapter = f"Chương {current_chapter_number}: {current_chapter_title}"
        # Loại bỏ dòng tiêu đề chương khỏi nội dung chunk nếu nó là dòng duy nhất hoặc rất ngắn
        # Hoặc nếu bạn muốn, có thể giữ lại một phần như một dấu hiệu bắt đầu chương.
        # Ở đây, chúng ta giả định là muốn loại bỏ hoàn toàn khỏi page_content nếu nó đứng riêng.
        # Nếu chunk_content chỉ chứa tiêu đề chương (ví dụ, sau khi split("\n\n"))
        lines_in_chunk = chunk_content.splitlines()
        if len(lines_in_chunk) > 0 and lines_in_chunk[0].strip() == f"Chương {current_chapter_number}: {current_chapter_title}":
            chunk_content = "\n".join(lines_in_chunk[1:]).strip() # Bỏ dòng tiêu đề
        elif len(lines_in_chunk) > 0 and lines_in_chunk[0].strip() == f"Chương {current_chapter_number}":
             chunk_content = "\n".join(lines_in_chunk[1:]).strip()


    elif chapter_match_simple and not chapter_match_title : # Nếu chỉ có "Chương X"
        current_chapter_number = int(chapter_match_simple.group(1))
        current_chapter = f"Chương {current_chapter_number}" # Không có tiêu đề cụ thể
        lines_in_chunk = chunk_content.splitlines()
        if len(lines_in_chunk) > 0 and lines_in_chunk[0].strip() == f"Chương {current_chapter_number}":
            chunk_content = "\n".join(lines_in_chunk[1:]).strip()

    # Nếu chunk quá ngắn sau khi bỏ tiêu đề, có thể bỏ qua hoặc gộp với chunk sau (logic phức tạp hơn)
    if not chunk_content.strip(): # Bỏ qua chunk rỗng
        continue

    doc_metadata = {
        "source": input_file_path, # Tên file nguồn
        "chunk_id": i,
    }
    if current_chapter:
        doc_metadata["chapter_info"] = current_chapter


    doc = Document(page_content=chunk_content, metadata=doc_metadata)
    documents.append(doc)

# --- In ra một vài Document đầu tiên để kiểm tra ---
print(f"\nTổng số Document được tạo ra: {len(documents)}")
print("\n--- 5 Document đầu tiên (sau khi xử lý tiêu đề chương): ---")
for i, doc in enumerate(documents[:5]):
    print(f"\n--- Document {i+1} ---")
    print(f"Content (first 150 chars): {doc.page_content[:150]}...")
    print(f"Metadata: {doc.metadata}")

# --- Lưu các Document objects đã xử lý ra file pickle ---
output_document_file = 'DSTS_documents_ch1-9.pkl'
with open(output_document_file, 'wb') as f:
    pickle.dump(documents, f)
print(f"\nCác Document objects đã được lưu vào file: {output_document_file}")
print(f"Hãy kiểm tra nội dung của các chunk đầu tiên và metadata để đảm bảo logic xác định chương hoạt động như mong đợi.")
print("Bạn có thể cần tinh chỉnh lại `chunk_size`, `chunk_overlap` hoặc logic xử lý tiêu đề chương.")

Đang xử lý và tạo Document objects cho 96 chunks thô...

Tổng số Document được tạo ra: 96

--- 5 Document đầu tiên (sau khi xử lý tiêu đề chương): ---

--- Document 1 ---
Content (first 150 chars): (Thèm ăn xiên nướng ở mấy hàng bán rong quá đi mất.)
Ngẩng đầu nhìn lên bầu trời âm u đầy mây mù, Miêu Miêu trút một tiếng thở dài.
Xung quanh cô là m...
Metadata: {'source': 'DSTS.txt', 'chunk_id': 0, 'chapter_info': 'Chương 1: Miêu Miêu'}

--- Document 2 ---
Content (first 150 chars): Còn đối với một dược sư có cuộc sống tương đối ổn như Miêu Miêu thì đây đúng là tay bay vạ gió.
Đối với Miêu Miêu, cô chẳng quan tâm lũ người này bắt ...
Metadata: {'source': 'DSTS.txt', 'chunk_id': 1, 'chapter_info': 'Chương 1: Miêu Miêu'}

--- Document 3 ---
Content (first 150 chars): Dù có là chốn lầu son gác tía nơi những bậc cao quý cư ngụ, hay là nơi ngõ liễu tường hoa xô bồ ngoài phố thị, đều không khác gì nhau.
Miêu Miêu ôm gi...
Metadata: {'source': 'DSTS.txt', 'chunk_id': 2, 'chapter_info': 'Chương 

In [2]:
import pickle
from langchain_community.embeddings import HuggingFaceEmbeddings # Sử dụng bản cập nhật
from langchain_community.vectorstores import FAISS # Sử dụng bản cập nhật
import os # Để kiểm tra file tồn tại

# --- Đường dẫn file và tên model ---
document_file_path = 'DSTS_documents_ch1-9.pkl'
# Chọn một model embedding phù hợp với tiếng Việt và tài nguyên của bạn
# "bkai-foundation-models/vietnamese-bi-encoder" là một lựa chọn tốt cho tiếng Việt
# "all-MiniLM-L6-v2" nhanh và nhẹ, nhưng có thể không tối ưu bằng cho tiếng Việt
embedding_model_name = "bkai-foundation-models/vietnamese-bi-encoder"
# embedding_model_name = "all-MiniLM-L6-v2"

# Đường dẫn để lưu FAISS index
faiss_index_output_path = "DSTS_faiss_index_ch1-9"

# --- 1. Tải lại các Document objects đã được chunk ---
if not os.path.exists(document_file_path):
    print(f"Lỗi: File '{document_file_path}' không tồn tại. Vui lòng chạy lại bước chia chunk.")
    exit()

print(f"Đang tải Document objects từ '{document_file_path}'...")
with open(document_file_path, 'rb') as f:
    documents = pickle.load(f)
print(f"Đã tải {len(documents)} Document objects.")

if not documents:
    print("Lỗi: Danh sách documents rỗng. Vui lòng kiểm tra lại file pickle.")
    exit()

# --- 2. Khởi tạo mô hình Embedding ---
print(f"Đang khởi tạo mô hình embedding: '{embedding_model_name}'...")
# Sử dụng 'cpu' nếu không có GPU hoặc GPU không đủ mạnh. Thay 'cuda' nếu có.
# Lần đầu tải model có thể mất chút thời gian.
try:
    embeddings_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={'device': 'cpu'} # Thay 'cuda' nếu bạn có GPU và muốn sử dụng
    )
    print("Mô hình embedding đã được khởi tạo.")
except Exception as e:
    print(f"Lỗi khi khởi tạo mô hình embedding: {e}")
    print("Hãy đảm bảo bạn đã cài đặt sentence-transformers và model name là chính xác.")
    print("Thử chạy: pip install sentence-transformers")
    exit()

# --- 3. Tạo Vector Store (FAISS) từ các documents và embeddings model ---
# FAISS sẽ tự động tạo embeddings cho các documents.
# Quá trình này có thể mất thời gian tùy thuộc vào số lượng chunk và sức mạnh máy tính.
print(f"\nĐang tạo FAISS index từ {len(documents)} documents...")
try:
    vector_store = FAISS.from_documents(documents, embeddings_model)
    print("FAISS index đã được tạo thành công.")
except Exception as e:
    print(f"Lỗi trong quá trình tạo FAISS index: {e}")
    # In ra một vài document đầu tiên để kiểm tra nội dung
    # for i, doc in enumerate(documents[:3]):
    #     print(f"Document {i} content type: {type(doc.page_content)}, metadata type: {type(doc.metadata)}")
    #     print(f"Document {i} page_content: {doc.page_content[:100]}")
    #     print(f"Document {i} metadata: {doc.metadata}")
    exit()


# --- 4. Lưu Vector Store ra file để sử dụng sau này ---
try:
    vector_store.save_local(faiss_index_output_path)
    print(f"FAISS index đã được lưu tại thư mục: '{faiss_index_output_path}'")
except Exception as e:
    print(f"Lỗi khi lưu FAISS index: {e}")
    exit()

# --- (Tùy chọn) Kiểm tra thử việc tìm kiếm ---
print("\n--- Thử nghiệm tìm kiếm tương đồng ---")
try:
    # Đảm bảo vector_store đã được tạo
    if 'vector_store' in locals() and vector_store:
        test_query = "Miêu Miêu làm gì ở hậu cung khi mới vào?"
        print(f"Tìm kiếm cho câu hỏi: '{test_query}'")
        # k=3 nghĩa là lấy 3 kết quả tương đồng nhất
        search_results = vector_store.similarity_search_with_score(test_query, k=3)

        if search_results:
            for i, (doc, score) in enumerate(search_results):
                print(f"\n--- Kết quả {i+1} (Score: {score:.4f}): ---")
                print(f"Trích đoạn: {doc.page_content[:250]}...") # In 250 ký tự đầu
                print(f"Metadata: {doc.metadata}")
        else:
            print("Không tìm thấy kết quả nào.")
    else:
        print("Vector store chưa được tạo để thực hiện tìm kiếm.")
except Exception as e:
    print(f"Lỗi trong quá trình tìm kiếm thử nghiệm: {e}")

print("\nHoàn thành bước tạo và lưu Vector Store!")

Đang tải Document objects từ 'DSTS_documents_ch1-9.pkl'...
Đã tải 96 Document objects.
Đang khởi tạo mô hình embedding: 'bkai-foundation-models/vietnamese-bi-encoder'...


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30216\3095876668.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


Mô hình embedding đã được khởi tạo.

Đang tạo FAISS index từ 96 documents...
FAISS index đã được tạo thành công.
FAISS index đã được lưu tại thư mục: 'DSTS_faiss_index_ch1-9'

--- Thử nghiệm tìm kiếm tương đồng ---
Tìm kiếm cho câu hỏi: 'Miêu Miêu làm gì ở hậu cung khi mới vào?'

--- Kết quả 1 (Score: 19.6825): ---
Trích đoạn: Khi Miêu Miêu nhắc đến chuyện mình đã được nghe Tiểu Lan kể, nữ quan đó liền vui vẻ nhận lời đối với cô.
Nơi hậu cung vốn không có chỗ cho tình yêu nam nữ cháy bỏng, nên xem ra ngay cả những kẻ không còn là đàn ông như hoạn quan cũng trở thành đối tư...
Metadata: {'source': 'DSTS.txt', 'chunk_id': 15, 'chapter_info': 'Chương 2: Hai vị phi tần'}

--- Kết quả 2 (Score: 21.1794): ---
Trích đoạn: Khi thời gian để tang kết thúc, các dải băng đen cũng dần biến mất khỏi tầm mắt, tin đồn về Ngọc Diệp phi lại nổi lên. Có vẻ sau khi mất đi Hoàng tử, Hoàng đế quá đau lòng nên đã dồn hết yêu thương cho tiểu công chúa còn sống sót.
Nhưng chẳng nghe th...
Metadata: {'source': 

In [4]:
import os
from dotenv import load_dotenv

load_dotenv() # Không cần thiết cho Ollama nhưng giữ lại nếu bạn có các key khác

# Import các lớp cần thiết từ LangChain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama # Sử dụng ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate # Để tùy chỉnh prompt
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Đường dẫn và tên model embedding (GIỮ NGUYÊN) ---
faiss_index_path = "DSTS_faiss_index_ch1-9"
embedding_model_name = "bkai-foundation-models/vietnamese-bi-encoder"

# --- 1. Tải lại FAISS index và khởi tạo embeddings model (GIỮ NGUYÊN) ---
print(f"Đang khởi tạo mô hình embedding: '{embedding_model_name}'...")
try:
    embeddings_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={'device': 'cpu'} # Hoặc 'cuda'
    )
    print("Mô hình embedding đã được khởi tạo.")
except Exception as e:
    print(f"Lỗi khi khởi tạo mô hình embedding: {e}")
    exit()

print(f"\nĐang tải FAISS index từ thư mục: '{faiss_index_path}'...")
if not os.path.exists(faiss_index_path):
    print(f"Lỗi: Thư mục FAISS index '{faiss_index_path}' không tồn tại.")
    exit()
try:
    vector_store = FAISS.load_local(faiss_index_path, embeddings_model, allow_dangerous_deserialization=True)
    print("FAISS index đã được tải thành công.")
except Exception as e:
    print(f"Lỗi khi tải FAISS index: {e}")
    exit()

# --- 2. Khởi tạo LLM (Sử dụng Ollama với Llama 3) ---
print("\nĐang khởi tạo LLM (Ollama - Llama 3)...")
# Đảm bảo Ollama server đang chạy và bạn đã pull model này
ollama_model_name = "llama3:8b-instruct-q4_0" # Hoặc model Llama 3 khác bạn đã pull

try:
    llm = ChatOllama(
        model=ollama_model_name,
        temperature=0.2,  # Giảm nhiệt độ để câu trả lời chính xác và bám context hơn
        # request_timeout=120 # Tăng timeout nếu model lớn và phản hồi chậm
    )
    print(f"LLM Ollama (model: {ollama_model_name}) đã được khởi tạo.")
except Exception as e:
    print(f"Lỗi khi khởi tạo Ollama LLM: {e}")
    print(f"Hãy đảm bảo Ollama đang chạy trên máy của bạn và model '{ollama_model_name}' đã được tải (ví dụ: ollama pull {ollama_model_name}).")
    exit()

# --- 3. (Tùy chọn nhưng KHUYẾN NGHỊ) Tùy chỉnh Prompt cho Llama ---
# Llama Instruct models thường hoạt động tốt với một định dạng prompt cụ thể.
# Bạn có thể thử nghiệm với prompt mặc định của RetrievalQA trước,
# nhưng nếu kết quả chưa tốt, hãy thử tùy chỉnh.

# Ví dụ một prompt template có thể phù hợp hơn với Llama Instruct:
# (Tham khảo thêm tài liệu của Llama 3 để có prompt tối ưu nhất)
prompt_template_str = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Bạn là một trợ lý AI hữu ích. Hãy sử dụng các đoạn thông tin sau đây để trả lời câu hỏi của người dùng.
Nếu bạn không biết câu trả lời dựa trên thông tin được cung cấp, hãy nói rằng bạn không biết. Đừng cố bịa ra câu trả lời.
Luôn trả lời bằng tiếng Việt.

Thông tin tham khảo:
{context}<|eot_id|><|start_header_id|>user<|end_header_id|>
Câu hỏi: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Câu trả lời hữu ích:"""

PROMPT = PromptTemplate(
    template=prompt_template_str, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

# --- 4. Tạo RetrievalQA chain ---
print("\nĐang tạo RetrievalQA chain...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # "stuff" hoạt động tốt nếu tổng context + question < giới hạn của Llama
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}), # Lấy 3 chunks
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs # Sử dụng prompt tùy chỉnh
)
print("RetrievalQA chain đã được tạo.")

# --- 5. Vòng lặp hỏi đáp (GIỮ NGUYÊN) ---
print("\n--- CHATBOT DƯỢC SƯ TỰ SỰ (với Llama 3 - Ollama) ---")
print("Gõ 'thoát' để kết thúc phiên trò chuyện.")
print("Gõ 'nguồn' sau câu trả lời để xem các đoạn văn bản tham khảo.")

last_response_sources = None

while True:
    user_query = input("\nBạn hỏi: ").strip()

    if user_query.lower() == 'thoát':
        break
    if user_query.lower() == 'nguồn':
        if last_response_sources:
            print("\n--- Nguồn tài liệu tham khảo cho câu trả lời trước: ---")
            for i, source_doc in enumerate(last_response_sources):
                print(f"\nNguồn {i+1} (Từ chunk có metadata: {source_doc.metadata}):")
                content_preview = source_doc.page_content.replace("\n", " ")[:300]
                print(f"   {content_preview}...")
            last_response_sources = None
        else:
            print("Chưa có câu trả lời nào trước đó để hiển thị nguồn.")
        continue

    if not user_query:
        continue

    print("Chatbot đang suy nghĩ (Llama 3 đang xử lý)...")
    try:
        result = qa_chain.invoke({"query": user_query})
        print("\nChatbot trả lời:")
        # ChatOllama thường trả về kết quả trong result['result'] hoặc trực tiếp là một AIMessage
        # Nếu result là AIMessage thì result.content là câu trả lời
        if isinstance(result.get("result"), str):
            print(result["result"])
        elif hasattr(result, 'content') and isinstance(result.content, str): # Cho trường hợp trả về AIMessage
             print(result.content)
        else:
            # In ra để debug nếu cấu trúc khác
            print(f"Kết quả thô từ chain: {result}")
            # Nếu "result" là một dictionary chứa message object, bạn có thể cần truy cập sâu hơn
            if isinstance(result.get("result"), dict) and "content" in result.get("result"):
                 print(result.get("result").get("content"))
            else:
                 print(f"Không thể trích xuất câu trả lời rõ ràng từ kết quả.")


        last_response_sources = result.get("source_documents")

    except Exception as e:
        print(f"Đã xảy ra lỗi khi gọi Llama 3: {e}")

print("\nCảm ơn bạn đã sử dụng chatbot! Tạm biệt.")

Đang khởi tạo mô hình embedding: 'bkai-foundation-models/vietnamese-bi-encoder'...
Mô hình embedding đã được khởi tạo.

Đang tải FAISS index từ thư mục: 'DSTS_faiss_index_ch1-9'...
FAISS index đã được tải thành công.

Đang khởi tạo LLM (Ollama - Llama 3)...
LLM Ollama (model: llama3:8b-instruct-q4_0) đã được khởi tạo.

Đang tạo RetrievalQA chain...
RetrievalQA chain đã được tạo.

--- CHATBOT DƯỢC SƯ TỰ SỰ (với Llama 3 - Ollama) ---
Gõ 'thoát' để kết thúc phiên trò chuyện.
Gõ 'nguồn' sau câu trả lời để xem các đoạn văn bản tham khảo.
Chatbot đang suy nghĩ (Llama 3 đang xử lý)...

Chatbot trả lời:
Theo thông tin được cung cấp, Miêu Miêu là một nô tì cấp bậc thấp nhất, chưa được nhận chức danh "nữ quan". Cô gái này có nước da rắn rỏi lấm tấm tàn nhang và tay chân khẳng khiu như cành củi khô. Miêu Miêu đang làm việc trong một phi tần hạ cấp và phải thu thập những nguyên liệu có thể dùng được từ ngăn kéo ngoài cùng trở đi.
Chatbot đang suy nghĩ (Llama 3 đang xử lý)...

Chatbot trả lời:
Theo